In [2]:
import numpy as np
import re
import string
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report

train_file = "train.data.txt"
dev_file = "dev.data.txt"
test_file = "test.data.txt"
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
def tokenize_tweet(string_data:str):
    wordnet_lemmatizer = WordNetLemmatizer()
    tokenized = nltk.RegexpTokenizer('\w+')
    data = string_data.replace('\n', '')
    data = data.lower()
    data = re.sub('https?://\S+|www\.\S+', '', data)
    data = re.sub('[%s]' % re.escape(string.punctuation), '', data)
    # data = [wordnet_lemmatizer.lemmatize(word) for word in data.split(' ')]
    data = ' '.join([i for i in data.split(' ') if i not in stopwords])
    return data

train_data = pd.read_csv('./%s.csv'%train_file,keep_default_na=False)
dev_data = pd.read_csv('./%s.csv'%dev_file,keep_default_na=False)
test_data = pd.read_csv('./%s.csv'%test_file,keep_default_na=False)
def preprocess_token(df, dropNa=True):
    data = df.copy()
    data['main_tweet'] = data['main_tweet'].fillna('')
    if dropNa:
        data.replace('', np.nan, inplace=True)
        data.dropna(subset=['main_tweet'], inplace=True)
    text = data['main_tweet'].apply(lambda x: tokenize_tweet(x))
    data['main_tweet'] = text
    return data


train_data=preprocess_token(train_data)
train_data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jiachen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jiachen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


main_tweet        main_tweet_id  \
0     5 regularly rinsing nose saline help prevent i...  1250219300389974016   
1      french police chief killed charliehebdo attack     554886875303780352   
2     coronavirus disease covid19 advice public\r\r✳...  1237901309011021825   
3     ottawa police confirm multiple suspects shooti...   524958128392376320   
4     primary focus government isnt alleviate suffer...  1239295488677085185   
...                                                 ...                  ...   
1889  4 cannot transmitted goods manufactured china ...  1237545128828342277   
1890  desperate ted cruz claims planned parenthood s...   671181758692507648   
1891  thoughts prayers enough pres obama speaks mass...   672513234419638273   
1892  police surrounded building suspected charliehe...   553508098825261056   
1894  lynnesimpkin help 👩‍🏫\r9am socialism  need bad...  1241082793737818113   

      verified  followers                                            replies  \
0        False        410  [{"tweet_id": 1250219116993974272, "tweet": "4...   
1         True    3229894  [{"tweet_id": 554959644125167617, "tweet": "De...   
2        False        613  [{"tweet_id": 1237901311439450112, "tweet": "I...   
3         True   19783124  [{"tweet_id": 524961934064754688, "tweet": "@W...   
4        False       4889                                                 []   
...        ...        ...                                                ...   
1889     False        631  [{"tweet_id": 1237545126278258703, "tweet": "#...   
1890      True     143090  [{"tweet_id": 671200376843067392, "tweet": "@B...   
1891      True   17449031  [{"tweet_id": 672513853645717504, "tweet": "@A...   
1892      True    9077962  [{"tweet_id": 553509546602553344, "tweet": "@N...   
1894     False       3103  [{"tweet_id": 1241041450084839424, "tweet": "E...   

          label  
0     nonrumour  
1        rumour  
2     nonrumour  
3     nonrumour  
4     nonrumour  
...         ...  
1889  nonrumour  
1890     rumour  
1891     rumour  
1892  nonrumour  
1894  nonrumour  

[1567 rows x 6 columns]

In [3]:
!pip install transformers
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_label = le.fit_transform(train_data['label'])
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=True)
train_text = train_data['main_tweet'].tolist()
encoding = tokenizer(train_text,truncation=True,max_length=100, padding=True, return_tensors="tf")


dev_data=preprocess_token(dev_data)
dev_text = dev_data['main_tweet'].tolist()
dev_encode = tokenizer(dev_text,truncation=True,max_length=100, padding=True, return_tensors="tf")
dev_label = le.transform(dev_data['label'])





model = TFAutoModel.from_pretrained(model_name, num_labels=2)

print(encoding["input_ids"].shape)

def create_model():
    input_ids = tf.keras.layers.Input(shape=(100,), name='input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(100,), name='attention_mask', dtype='int32')

    embeddings = model(input_ids, attention_mask=mask)[0]
#     X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
    X = tf.keras.layers.LSTM(128)(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.1)(X)
    y = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(X)

    tfmodel = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    return tfmodel

tfmodel = create_model()
optimizer = tf.keras.optimizers.Adam(2e-5)
tfmodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = tfmodel.fit({"input_ids": encoding["input_ids"], 'attention_mask': encoding["attention_mask"]}, \
                      train_label,\
                      validation_data=({"input_ids": dev_encode["input_ids"], 'attention_mask': dev_encode["attention_mask"]}, dev_label),\
                      epochs=10)
# tfmodel.load_weights('bert-weight.h5')

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


(1567, 100)
Epoch 1/10
49/49 [==============================] - 45s 523ms/step - loss: 0.5118 - accuracy: 0.7403 - val_loss: 0.5075 - val_accuracy: 0.8523
Epoch 2/10
49/49 [==============================] - 22s 447ms/step - loss: 0.2417 - accuracy: 0.9138 - val_loss: 0.3863 - val_accuracy: 0.9084
Epoch 3/10
49/49 [==============================] - 22s 453ms/step - loss: 0.1417 - accuracy: 0.9636 - val_loss: 0.2889 - val_accuracy: 0.9271
Epoch 4/10
49/49 [==============================] - 22s 450ms/step - loss: 0.0674 - accuracy: 0.9879 - val_loss: 0.2076 - val_accuracy: 0.9383
Epoch 5/10
49/49 [==============================] - 22s 448ms/step - loss: 0.0387 - accuracy: 0.9943 - val_loss: 0.1620 - val_accuracy: 0.9402
Epoch 6/10
49/49 [==============================] - 22s 454ms/step - loss: 0.0249 - accuracy: 0.9974 - val_loss: 0.1794 - val_accuracy: 0.9364
Epoch 7/10
49/49 [==============================] - 22s 455ms/step - loss: 0.0160 - accuracy: 0.9994 - val_loss: 0.1559 - val_accu

In [1]:
import json
import pandas as pd
import tweepy
import json
import pandas as pd
import re
import nltk
from textblob import TextBlob


api_key = "aJTgi4d1H1zmQNkQuHeualNhP"
api_secrets = "yPFdp2Bbib25r1iPQCqBuPq8B9UzoJbIdpv1jgzEZFEg1eJl6X"
access_token = "1409382627124019204-oPpzVGuCwyFfQTfoocFhgHi68whhog"
access_secret = "esgwpH5gnRfNB0SpiHkO52mZSI5VKKnn8SG2pafEnzj2b"
 
# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
 
api = tweepy.API(auth)

In [3]:
def get_tweets_from_api(ids):
    for i in range(0, len(ids), 100):
        split = ids[i:i+100]
        for tweet in api.lookup_statuses(split, tweet_mode='extended'):
            tweets_dict[tweet.id_str] = {}
            tweets_dict[tweet.id_str]['text'] = tweet.full_text
            tweets_dict[tweet.id_str]['user'] = tweet.user.screen_name
            tweets_dict[tweet.id_str]['verified'] = tweet.user.verified
            tweets_dict[tweet.id_str]['followers'] = tweet.user.followers_count
            tweets_dict[tweet.id_str]['num_statuses'] = tweet.user.statuses_count
            tweets_dict[tweet.id_str]['likes'] = tweet.favorite_count
            tweets_dict[tweet.id_str]['retweets'] = tweet.retweet_count
            hashtag = tweet.entities['hashtags']
            if hashtag == []:
                tweets_dict[tweet.id_str]['hashtag'] = ""
            else:
                tweets_dict[tweet.id_str]['hashtag'] = hashtag[0]['text']

                    
source_ids = []
file = open(r"E:\Capstone\Twitter-Rumour-Detection-main\Data")
threads = file.readlines()
file.close()
source_ids = [thread.replace('\n', '').split(',')[0] for thread in threads]

tweets_dict = {}

get_tweets_from_api(source_ids)
                      
tweet_text = []
tweet_user = []
tweet_verified = []
tweet_likes = []
tweet_retweets = []
tweet_hashtag = []
tweet_follow = []
tweet_statuses = []

for tweet_id in tweets_dict.keys():
    tweet_text.append(tweets_dict[tweet_id]['text'])
    tweet_user.append(tweets_dict[tweet_id]['user'])
    tweet_verified.append(tweets_dict[tweet_id]['verified'])
    tweet_likes.append(tweets_dict[tweet_id]['likes'])
    tweet_retweets.append(tweets_dict[tweet_id]['retweets'])
    tweet_hashtag.append(tweets_dict[tweet_id]['hashtag'])
    tweet_follow.append(tweets_dict[tweet_id]['followers'])
    tweet_statuses.append(tweets_dict[tweet_id]['num_statuses'])
    
covid_ids_df = pd.DataFrame({'id': tweets_dict.keys(), 
                             'text': tweet_text, 
                             'user': tweet_user,
                             'verified': tweet_verified,
                             'likes': tweet_likes,
                             'retweets': tweet_retweets,
                             'hashtag': tweet_hashtag,
                             'followers': tweet_follow,
                             'statuses': tweet_statuses},
                              index=tweets_dict.keys())

covid_ids_df.to_csv('covid_id_df.csv', header=True)

PermissionError: [Errno 13] Permission denied: 'E:\\Capstone\\Twitter-Rumour-Detection-main\\Data'

In [4]:
# tfmodel.save("modelbert.h5")
# dev_data=preprocess_token(dev_data)

dev_text = dev_data['main_tweet'].tolist()
dev_encode = tokenizer(dev_text,truncation=True,max_length=100, padding=True, return_tensors="tf")
dev_label = le.transform(dev_data['label'])
tfmodel.evaluate({"input_ids": dev_encode["input_ids"], 'attention_mask': dev_encode["attention_mask"]}, dev_label)

17/17 [==============================] - 2s 142ms/step - loss: 0.1684 - accuracy: 0.9421


[0.1684463620185852, 0.9420560598373413]

In [5]:
predictionzz = tfmodel.predict({"input_ids": dev_encode["input_ids"], 'attention_mask': dev_encode["attention_mask"]})
predictionzz = (predictionzz > 0.5).astype("int32")
predictionzz = np.ndarray.flatten(predictionzz)
print("Bert on dev set")
print(classification_report(dev_label, predictionzz, digits=4))

Bert on dev set
              precision    recall  f1-score   support

           0     0.9451    0.9833    0.9638       420
           1     0.9286    0.7913    0.8545       115

    accuracy                         0.9421       535
   macro avg     0.9368    0.8873    0.9091       535
weighted avg     0.9415    0.9421    0.9403       535



In [6]:
##create Bert for test 2e5
test_data=preprocess_token(test_data)

test_text = test_data['main_tweet'].tolist()
test_encode = tokenizer(test_text,truncation=True,max_length=100, padding=True, return_tensors="tf")

prediction = tfmodel.predict({"input_ids": test_encode["input_ids"], 'attention_mask': test_encode["attention_mask"]})
prediction = (prediction > 0.5).astype("int32")
prediction = np.ndarray.flatten(prediction)
pd.DataFrame({"Predicted":  prediction}).to_csv('BertLSTM+Feedforward-submit.csv', index_label="Id")